In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from arcgis.geometry import Geometry, SpatialReference, Point, Polygon
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
import os
import datetime as dt
from datetime import datetime

In [ ]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
find_storymaps = gis.content.search("title:* Community Storymap NOT Census", item_type="Web Mapping Application", max_items=485, outside_org=False)

In [ ]:
#Set up a log file to write errors to
today = [dt.date.today()]
log_path = r"T:\CDO_To_Esri\Storymaps\_Archive\Logfiles\Municipalities\Update General Services"
log_file = os.path.join(log_path, "Update General Services_" + str(today[0]) + str(".txt"))
out = open(log_file,'w')

for f in find_storymaps:
    try:
        print(f.title, f.id)
        if f.title != 'Alaska DCRA Community Storymaps' and 'Haines Borough' not in str(f.title) and not f.title.startswith('Communit') and 'Community' in str(f.title):
            csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
            sections = csmap.properties["values"]["story"]["sections"]
            panel_gensvs = sections[8]
            community = f.title.replace(" Community Storymap", "").replace(" (Unincorporated Community)", "").replace("'","''")
            if community.startswith("City & Borough of "):
                community = community.replace("City & Borough of ", "")
            if community.startswith("Municipality of "):
                community = community.replace("Municipality of ","")
            print(community)
            filter_ID = "CommunityName = '" + str(community.replace("'","''")) + "'"
            query = "CommunityName = '" + str(community.replace("'","''")) + "'"

    #Healthcare tables
            healthcare = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/Services_Healthcare_SafetyNet_Directory/MapServer/0')
            healthcare_fts = healthcare.query(where=filter_ID, out_fields="*").df
            if str(healthcare_fts) == 'Empty DataFrame\nColumns: []\nIndex: []':
                embed_hlth_tbl = 'There are no healthcare facilities data for this community.'
            else:
                print("Creating healthcare facilities table...")
                tbl_lst = ['<table border="1"><tr><th width="30%">Healthcare Facility</th><th width="30%">Services Provided</th><th width="40%">Address</th>']
                for h in healthcare_fts.EntityName:
                    hc_data = healthcare.query(where="EntityName = '" + str(h.replace("'","''")) +"'", out_fields = "*").df
                    facility = hc_data.EntityName[0]
                    address = str(hc_data.MailingAddress[0]) + ", " + str(hc_data.City[0]) + ", " + str(hc_data.State[0])
                    services = [hc_data.FacilityType1[0],hc_data.FacilityType2[0],hc_data.FacilityType3[0],hc_data.FacilityType4[0], hc_data.FacilityType5[0]]
                    svcs_lst = (str(services).replace(", None", " ")).replace('[',"").replace(']',"").replace("'","")
                    tbl_lst.append('<tr><td><span style="font-size:14px">' + str(facility) + '</span></td><td><span style="font-size:14px">' + str(svcs_lst) + '</span></td><td><span style="font-size:14px">' + str(address) + '</span></td></tr>')
                embed_hlth_tbl = ' '.join(tbl_lst)

            #Public libraries
            pl_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Services/MapServer/1')
            if str(pl_base.query(where=query, out_fields="*").df) == 'Empty DataFrame\nColumns: []\nIndex: []':
                pub_lib_tbl = "There are no public libraries in " + str(community) + "."
            else:
                libraries = pl_base.query(where=query, out_fields="*").df.EntityName
                pl_tbl = ['<table border="1"><th width="40%">Library Name</th><th width="30%">Address</th><th width="20%">Phone</th><th width="15%">Email</th><tbody>']
                libraries = pl_base.query(where=query, out_fields="*").df.EntityName
                pl_tbl = ['<table border="1"><th width="40%">Library Name</th><th width="30%">Address</th><th width="20%">Phone</th><th width="15%">Email</th><tbody>']
                for p in libraries:
                    lib = pl_base.query(where="EntityName = '" + str(p.replace("'","''")) +"'", out_fields="*").df.EntityName[0]
                    lib_address = pl_base.query(where="EntityName = '" + str(p.replace("'","''")) +"'", out_fields="*").df.Address[0]
                    lib_phone = pl_base.query(where="EntityName = '" + str(p.replace("'","''")) +"'", out_fields="*").df.Phone[0]
                    lib_email = pl_base.query(where="EntityName = '" + str(p.replace("'","''")) +"'", out_fields="*").df.Email[0]
                    pl_tbl.append('<tr><td><span style="font-size:14px">' + str(lib) + '</span></td><td><span style="font-size:14px">' + str(lib_address) + '</span></td><td><span style="font-size:14px">' + str(lib_phone) + '</span></td><td><span style="font-size:14px">' + str(lib_email) + '</span></td></tr>')
                pl_tbl.append('</tbody></table>')
                pub_lib_tbl = ' '.join(pl_tbl)

            #Post offices
            post_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Services/MapServer/0')
            if str(post_base.query(where=query,out_fields="*").df) == 'Empty DataFrame\nColumns: []\nIndex: []':
                embed_post = 'There are no post offices operating in ' + str(community) + '.'
            else:
                offices = post_base.query(where=query,out_fields="*").df.EntityName
                post_tbl = ['<table border="1"><th width="30%">Post Office Name</th><th width="30%">Address</th><th width="20%">Phone</th><th width="20%">Email</th><tbody>']
                for p in set(offices):
                    post = post_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.EntityName[0]
                    post_address = post_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.Address[0]
                    post_phone = post_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.Phone[0]
                    post_email = post_base.query(where="EntityName = '" + str(p) +"'", out_fields="*").df.Email[0]
                    post_tbl.append('<tr><td><span style="font-size:14px">' + str(post) + '</span></td><td><span style="font-size:14px">' + str(post_address) + '</span></td><td><span style="font-size:14px">' + str(post_phone) + '</span></td><td><span style="font-size:14px">' + str(post_email) + '</span></td></tr>')
                post_tbl.append('</tbody></table>')
                embed_post = ' '.join(post_tbl)
  
            #Update the content using tables above
            
            panel_gensvs["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p><p><i>The map at the right shows the locations of general services in this community. Click the map icons to see more information about each facility. You can also zoom in and out, and pan across the map to find other nearby facilities.</i></p><p><b>HEALTHCARE:</b></p><div><p>' + str(embed_hlth_tbl) + '</table></p></div><p>&nbsp;</p><p><b>POST OFFICES: </b></p><div>' + str(embed_post) + '</div><p>&nbsp;</p><p><b>PUBLIC LIBRARIES: </b></p><div><p>' + str(pub_lib_tbl) + '</p></div>'

            csmap.save()
            print("Services content updated.")
    except Exception as e:
        print("-------ISSUE WITH " + str(f.title) + "-------- " + str(e))
        out.write("General Services panel for " + str(f.title) +  " not properly updated. " + str(e))
        out.write('\n\n')
        print("Community written to error logfile.")
        continue
out.close()